In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers datasets accelerate

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.4 MB/s eta 0:00:00


In [ ]:
train_path = "/content/drive/MyDrive/CosmopediaStories/cosmopedia_stories_translated_tr_02.csv"
train_data = pd.read_csv(train_path)

val_path = "/content/drive/MyDrive/Bitirme/Datasets/validation_dataset.csv"
#val_path = "/content/drive/MyDrive/Bitirme/Datasets/large_validation.csv"
val_data = pd.read_csv(val_path)

n_total = len(train_data)
chunk_size = n_total // 4


train_texts = train_data["Text"].tolist()
train_dataset = Dataset.from_dict({"text": train_texts})

val_texts = val_data["Text"].tolist()
val_dataset = Dataset.from_dict({"text": val_texts})

In [ ]:
train_path = "/content/drive/MyDrive/CosmopediaStories/cosmopedia_stories_translated_tr_02.csv"
train_data = pd.read_csv(train_path)

val_path = "/content/drive/MyDrive/Bitirme/Datasets/validation_dataset.csv"
#val_path = "/content/drive/MyDrive/Bitirme/Datasets/large_validation.csv"
val_data = pd.read_csv(val_path)

n_total = len(train_data)
chunk_size = n_total // 50

train_data = train_data.iloc[0:chunk_size*5]

train_data = train_data.iloc[0:chunk_size*5]
#train_data = train_data.iloc[chunk_size*5:chunk_size*10]
#train_data = train_data.iloc[chunk_size*10:chunk_size*15]
#train_data = train_data.iloc[chunk_size*15:chunk_size*20]

train_texts = train_data["Text"].tolist()
train_dataset = Dataset.from_dict({"text": train_texts})

val_texts = val_data["Text"].tolist()
val_dataset = Dataset.from_dict({"text": val_texts})

In [ ]:
# Step 2: Load Tokenizer and Model
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "unsloth/Llama-3.2-1B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
).to(device)

In [ ]:
# Step 4: Tokenization
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )

tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

Map:   0%|          | 0/22880 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
# Step 4: Create DataLoaders
train_loader = DataLoader(tokenized_train, batch_size=8, shuffle=True, collate_fn=lambda x: {
    'input_ids': torch.stack([torch.tensor(f['input_ids']) for f in x]),
    'attention_mask': torch.stack([torch.tensor(f['attention_mask']) for f in x]),
    'labels': torch.stack([torch.tensor(f['input_ids']) for f in x])
})

val_loader = DataLoader(tokenized_val, batch_size=8, shuffle=True, collate_fn=lambda x: {
    'input_ids': torch.stack([torch.tensor(f['input_ids']) for f in x]),
    'attention_mask': torch.stack([torch.tensor(f['attention_mask']) for f in x]),
    'labels': torch.stack([torch.tensor(f['input_ids']) for f in x])
})

In [ ]:
# Step 5: Setup Optimizer
optimizer = AdamW(model.parameters(), lr=1e-6)
print(f"Total transformer layers: {len(model.model.layers)}")
print(next(model.parameters()).dtype)
print(model.dtype)

Total transformer layers: 16
torch.float32
torch.float32


In [ ]:
for i, layer in enumerate(model.model.layers):
    for param in layer.parameters():
        param.requires_grad = False

In [ ]:
layer_group = 1

for i, layer in enumerate(model.model.layers):
    if layer_group == 1 and i < 4:
        for param in layer.parameters():
            param.requires_grad = True
    elif layer_group == 2 and i >= 4 and i < 8:
        for param in layer.parameters():
            param.requires_grad = True
    elif layer_group == 3 and i >= 8 and i < 12:
        for param in layer.parameters():
            param.requires_grad = True
    elif layer_group == 4 and i >= 12 and i < 16:
        for param in layer.parameters():
            param.requires_grad = True

for i, layer in enumerate(model.model.layers):
    trainable = any(p.requires_grad for p in layer.parameters())
    status = "✅" if trainable else "❌"
    print(f"Layer {i}: {status}")

Layer 0: ✅
Layer 1: ✅
Layer 2: ✅
Layer 3: ✅
Layer 4: ❌
Layer 5: ❌
Layer 6: ❌
Layer 7: ❌
Layer 8: ❌
Layer 9: ❌
Layer 10: ❌
Layer 11: ❌
Layer 12: ❌
Layer 13: ❌
Layer 14: ❌
Layer 15: ❌


In [ ]:
for i, layer in enumerate(model.model.layers):
    if i == 4:
        for param in layer.parameters():
            param.requires_grad = True

for i, layer in enumerate(model.model.layers):
    trainable = any(p.requires_grad for p in layer.parameters())
    status = "✅" if trainable else "❌"
    print(f"Layer {i}: {status}")

Layer 0: ✅
Layer 1: ✅
Layer 2: ✅
Layer 3: ✅
Layer 4: ✅
Layer 5: ✅
Layer 6: ❌
Layer 7: ❌
Layer 8: ❌
Layer 9: ❌
Layer 10: ❌
Layer 11: ❌
Layer 12: ❌
Layer 13: ❌
Layer 14: ❌
Layer 15: ❌


In [ ]:
from torch.amp import GradScaler, autocast
from tqdm import tqdm

# To Save Steps
step_flag = 0

# Create gradient scaler
scaler = GradScaler()
num_epochs = 3
j = 0

total_steps = len(train_loader) * num_epochs
save_steps = [int(total_steps * i / 8) for i in range(1, 8)]
current_step = 0  # Toplam adım sayac

# Training loop with mixed precision
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")

    for batch in progress_bar:
        optimizer.zero_grad()
        batch = {k: v.to(device) for k, v in batch.items()}

        # Use autocast for mixed precision
        with autocast(device_type="cuda"):
            outputs = model(**batch)
            loss = outputs.loss

        # Scale loss and call backward
        scaler.scale(loss).backward()

        # Unscale gradients and clip them
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Step optimizer and update scaler
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()
        progress_bar.set_postfix({"Loss": loss.item()})

        current_step += 1

        # 🔒 Ara kaydetme noktası
        if current_step in save_steps and step_flag == 1:
            j += 1
            checkpoint_dir = f"/content/drive/MyDrive/Bitirme/Models/Checkpoints/1GbTest/llama-1b-base-cosmopedia-1gb.step_{j}"
            model.save_pretrained(checkpoint_dir)
            tokenizer.save_pretrained(checkpoint_dir)
            print(f"🔒 Checkpoint saved at step {current_step}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} Train Loss: {avg_loss:.4f}")

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    val_progress_bar = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")

    with torch.no_grad():  # No need to compute gradients for validation
        for batch in val_progress_bar:
            batch = {k: v.to(device) for k, v in batch.items()}

            # Use autocast for mixed precision during validation
            with autocast(device_type="cuda"):
                outputs = model(**batch)
                val_loss = outputs.loss

            total_val_loss += val_loss.item()
            val_progress_bar.set_postfix({"Val Loss": val_loss.item()})

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch {epoch+1} Validation Loss: {avg_val_loss:.4f}")

final_path = "/content/drive/MyDrive/Bitirme/Models/llama-1b-base-cosmopedia-3.1"
model.save_pretrained(final_path)
tokenizer.save_pretrained(final_path)
#torch.save(optimizer.state_dict(), final_path)
#torch.save(model.state_dict(), final_path)

Epoch 1/3: 100%|██████████| 2860/2860 [13:19<00:00,  3.58it/s, Loss=1.74]


Epoch 1 Train Loss: 1.5482


Validation Epoch 1/3: 100%|██████████| 1/1 [00:00<00:00, 31.58it/s, Val Loss=2.45]


Epoch 1 Validation Loss: 2.4486


Epoch 2/3: 100%|██████████| 2860/2860 [13:19<00:00,  3.58it/s, Loss=1.49]


...

Epoch 3/3: 100%|██████████| 2860/2860 [13:19<00:00,  3.58it/s, Loss=1.27]


Epoch 3 Train Loss: 1.3336


Validation Epoch 3/3: 100%|██████████| 1/1 [00:00<00:00, 31.62it/s, Val Loss=2.45]


Epoch 3 Validation Loss: 2.4491


('/content/drive/MyDrive/Bitirme/Models/llama-1b-base-cosmopedia-3.1/tokenizer_config.json',
 '/content/drive/MyDrive/Bitirme/Models/llama-1b-base-cosmopedia-3.1/special_tokens_map.json',
 '/content/drive/MyDrive/Bitirme/Models/llama-1b-base-cosmopedia-3.1/tokenizer.json')